# Baysian Predictive Synthesis

## abstract

 Spatial data are characterized by their spatial dependence, which is often complex, non-linear, and difficult to capture with a single model. Significant levels of model uncertainty -- arising from these characteristics -- cannot be resolved by model selection or simple ensemble methods. We address this issue by proposing a novel methodology that captures spatially varying model uncertainty, which we call Bayesian spatial predictive synthesis. Our proposal is derived by identifying the theoretically best approximate model under reasonable conditions, which is a latent factor spatially varying coefficient model in the Bayesian predictive synthesis framework. We then show that our proposed method produces exact minimax predictive distributions, providing finite sample guarantees. Two MCMC strategies are implemented for full uncertainty quantification, as well as a variational inference strategy for fast point inference. We also extend the estimation strategy for general responses. Through simulation examples and two real data applications, we demonstrate that our proposed spatial Bayesian predictive synthesis outperforms standard spatial models and advanced machine learning methods in terms of predictive accuracy.

## Ref

- [http://arxiv.org/abs/2203.05197](http://arxiv.org/abs/2203.05197)

In [1]:
import numpy as np
from numpy.random import default_rng
from copy import copy

# visualize
import arviz
import matplotlib.pyplot as plt

# cmdstanpy
from cmdstanpy import CmdStanModel

# random generator
rg = default_rng(123)

## Method

$$
y_i = \beta_{0i} + \sum_{j = 1}^{J} \beta_{ji}f_{ij} + \epsilon_{i} \quad \epsilon_i \sim N(0, \sigma^2)
$$

$$
\boldsymbol{\beta}_j  = (\beta_{j1}, \ldots \beta_{jn})^T \sim N(0, \tau_j G(g_j))
$$

In [2]:
# data settings
N = 1000
J = 5

# parameters
tau = [0.1, 0.2, 0.3, 0.4, 0.5]
sigma = 2.0
all_beta = np.array(
    [rg.multivariate_normal(np.zeros(N), tau[j] * np.eye(N)) for j in range(J)]
).T

f_matrix = rg.normal(size=(N, J))
Y = (all_beta * f_matrix).sum(axis=1) + rg.normal(loc=0, scale=sigma, size=N)

In [3]:
# モデル読み込み
model = CmdStanModel(stan_file="../../src/stan/BaysianPredictSynthesis.stan")

# モデル式
print("-------------------")
model.format()

ValueError: no such file /home/akihiro/project/note/src/stan/BaysianPredictSynthesis.stan

In [ ]:
# Data dict for stan
stan_data = {"N": N, "J": J, "y_obs": Y, "f_matrix": f_matrix}

# 計算
res = model.sample(
    data=stan_data,
    iter_warmup=1500,
    iter_sampling=500,
    parallel_chains=4,
    chains=4,
)

In [ ]:
arviz.plot_trace(
    res,
    var_names=["~beta"],
    legend=True,
    divergences=None,
    backend_kwargs={"constrained_layout": True},
)